https://www.thesportsdb.com/table.php?l=4334&s=2025-2026

In [6]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd

In [8]:
url = "https://v3.football.api-sports.io/standings"
params = {
    "league": 61,  # Ligue 1
    "season": 2024
}

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": API_HOST
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    standings = data['response'][0]['league']['standings'][0]

    # Transformer en DataFrame pour lecture facile
    df = pd.DataFrame([
        {
            "Position": team['rank'],
            "Équipe": team['team']['name'],
            "Points": team['points'],
            "Joués": team['all']['played'],
            "Gagnés": team['all']['win'],
            "Nuls": team['all']['draw'],
            "Perdus": team['all']['lose'],
            "Diff. Buts": team['goalsDiff']
        }
        for team in standings
    ])

    print(df)
else:
    print(f"Erreur {response.status_code}: {response.text}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:
API_KEY = "123"  # Clé publique par défaut
LEAGUE_ID = "4334"
SEASON = "2025-2026"

url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/lookuptable.php?l={LEAGUE_ID}&s={SEASON}"

print(f"🔍 Requête vers : {url}")

try:
    response = requests.get(url)
    response.raise_for_status()  # Lève une exception si erreur HTTP

    # Vérifie si le contenu est vide
    if not response.text.strip():
        print("⚠️ Réponse vide de l'API.")
    else:
        try:
            data = response.json()
            if "table" in data and data["table"]:
                print("✅ Données récupérées avec succès :\n")
                for team in data["table"]:
                    print(f"{team['intRank']}. {team['strTeam']} - {team['intPoints']} pts")
            else:
                print("⚠️ Pas de données de classement trouvées pour cette saison.")
        except ValueError:
            print("⚠️ Réponse non JSON. Voici le contenu brut :")
            print(response.text[:500])  # affiche les 500 premiers caractères
except requests.RequestException as e:
    print(f"🚨 Erreur de requête : {e}")

🔍 Requête vers : https://www.thesportsdb.com/api/v1/json/123/lookuptable.php?l=4334&s=2025-2026
✅ Données récupérées avec succès :

1. Paris SG - 21 pts
2. Lens - 20 pts
3. Marseille - 18 pts
4. Lyon - 18 pts
5. Lille - 17 pts


In [5]:
def get_team_rank(team_name):
    """Récupère le rang d'une équipe dans le classement de TheSportsDB."""
    url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/lookuptable.php?l={LEAGUE_ID}&s={SEASON}"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()

    if "table" not in data or not data["table"]:
        raise ValueError("Aucune donnée de classement trouvée.")

    for team in data["table"]:
        if team_name.lower() in team["strTeam"].lower():
            return int(team["intRank"])
    raise ValueError(f"Équipe '{team_name}' non trouvée dans la ligue.")

def proba_victoire(rang_A, rang_B):
    """Calcule les probabilités de victoire basées sur le classement."""
    pA = rang_B / (rang_A + rang_B)
    pB = rang_A / (rang_A + rang_B)
    return round(pA, 2), round(pB, 2)

def predire_match(equipe_A, equipe_B):
    """Prédit le vainqueur d’un match selon les rangs du classement."""
    rang_A = get_team_rank(equipe_A)
    rang_B = get_team_rank(equipe_B)

    pA, pB = proba_victoire(rang_A, rang_B)
    prediction = equipe_A if pA > pB else equipe_B

    print(f"Match : {equipe_A} (#{rang_A}) vs {equipe_B} (#{rang_B})")
    print(f"Probabilité {equipe_A} : {pA*100}% | {equipe_B} : {pB*100}%")
    print(f"Cotes {equipe_A} : {round(1/pA, 2)} | {equipe_B} : {round(1/pB, 2)}")
    print(f"➡️ Pronostic : victoire de {prediction}")

# Exemple d'utilisation :
predire_match("Auxerre", "Marseille")

ValueError: Équipe 'Auxerre' non trouvée dans la ligue.

In [7]:
import requests

API_KEY = "3"
LEAGUE_ID = "4334"   # Premier League
SEASON = "2025-2026"

def get_team_rank(team_name):
    """
    Récupère le rang d'une équipe depuis le classement TheSportsDB.
    """
    url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/lookuptable.php?l={LEAGUE_ID}&s={SEASON}"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()

    if "table" not in data or not data["table"]:
        raise ValueError("Aucune donnée de classement trouvée pour cette saison.")

    for team in data["table"]:
        if team_name.lower() in team["strTeam"].lower():
            return int(team["intRank"])

    raise ValueError(f"Équipe '{team_name}' non trouvée dans la ligue.")

def proba_victoire_avec_nul(rang_A, rang_B):
    """
    Calcule les probabilités de victoire et de nul basées uniquement sur le classement.
    """
    # Étape 1 : Probabilité brute selon le classement
    pA = rang_B / (rang_A + rang_B)
    pB = rang_A / (rang_A + rang_B)

    # Étape 2 : Probabilité du nul (plus les équipes sont proches, plus le nul est probable)
    diff = abs(rang_A - rang_B)
    pN = 0.25 - (diff / 100)  # base 25%, diminue si gros écart
    pN = max(0.05, min(pN, 0.30))  # bornes : 5% à 30%

    # Étape 3 : Normalisation pour que la somme fasse 1
    total = pA + pB
    pA = pA / total * (1 - pN)
    pB = pB / total * (1 - pN)

    return round(pA, 3), round(pN, 3), round(pB, 3)

def pronostic(equipe_A, equipe_B):
    """
    Calcule les probabilités et fait un pronostic entre deux équipes
    en fonction de leur classement réel TheSportsDB.
    """
    rang_A = get_team_rank(equipe_A)
    rang_B = get_team_rank(equipe_B)
    pA, pN, pB = proba_victoire_avec_nul(rang_A, rang_B)

    print(f"⚽ {equipe_A} (#{rang_A}) vs {equipe_B} (#{rang_B})")
    print(f"Probabilités : {equipe_A} gagne {pA*100:.1f}% | Nul {pN*100:.1f}% | {equipe_B} gagne {pB*100:.1f}%")
    print(f"Cotes : {equipe_A} gagne {1/pA:.2f} | Nul {1/pN:.2f} | {equipe_B} gagne {1/pB:.2f}")

    # Choix du pronostic
    if max(pA, pN, pB) == pA:
        prono = f"Victoire de {equipe_A}"
    elif max(pA, pN, pB) == pB:
        prono = f"Victoire de {equipe_B}"
    else:
        prono = "Match nul"

    print(f"➡️ Pronostic : {prono}\n")

# Exemple : PSG vs Marseille
pronostic("Paris SG", "Marseille")


⚽ Paris SG (#1) vs Marseille (#2)
Probabilités : Paris SG gagne 50.7% | Nul 24.0% | Marseille gagne 25.3%
Cotes : Paris SG gagne 1.97 | Nul 4.17 | Marseille gagne 3.95
➡️ Pronostic : Victoire de Paris SG

